
## fetch comments 

In [5]:
import os
import requests
import pandas as pd
from datetime import datetime
from tqdm import tqdm

# Reddit API credentials
client_id = 'AIgVuy1vju3sPYddN7fZYA'
secret_token = '4vS8QLhVdf8J_v4xaSgnTj7k1Sl9tg'

###########
client_id_2='I6urQepdb8l2A4pFs4ygKg'
secret_token_2='vftB0xazPmwYoy6AeLM-Ie-aunPz4Q'
username_2='TalkThese2972'
password_2='r4dNp2L7#@-phAb'
#################
username = 'BriefWall2565'
password = 'C@Jq,Byn@mGi-6F'

# Parameters
limit = 100  # Adjust the number of posts per request
num_iter = 10  # Adjust the number of iterations per subreddit




# Authenticate API and get OAuth token
def authenticate():
    client_auth = requests.auth.HTTPBasicAuth(client_id_2, secret_token_2)
    auth_data = {'grant_type': 'password', 'username': username_2, 'password': password_2}
    headers = {'User-Agent': 'myBot/0.0.1'}

    auth_res = requests.post('https://www.reddit.com/api/v1/access_token',
                             auth=client_auth, data=auth_data, headers=headers)
    if auth_res.status_code == 200:
        token = f"bearer {auth_res.json().get('access_token')}"
        headers['Authorization'] = token
        return headers
    else:
        raise Exception(f"Authentication failed! Status: {auth_res.status_code}, Message: {auth_res.text}")

def fetch_comments(headers, subreddit, post_id):
    """
    Fetches comments for a given Reddit post with detailed features.
    
    :param headers: Authentication headers
    :param subreddit: The subreddit name
    :param post_id: The ID of the post
    :return: DataFrame containing comments with detailed features
    """
    url = f"https://oauth.reddit.com/r/{subreddit}/comments/{post_id}"
    params = {'limit': 100}  # Adjust the limit as needed

    res = requests.get(url, headers=headers, params=params)
    if res.status_code != 200:
        print(f"Failed to fetch comments for post {post_id} in r/{subreddit}: {res.status_code}")
        return pd.DataFrame()

    try:
        response_json = res.json()
        if not isinstance(response_json, list) or len(response_json) < 2:
            print(f"Unexpected response structure: {response_json}")
            return pd.DataFrame()

        comments = []
        for comment in response_json[1]['data']['children']:
            if 'data' in comment:
                comment_data = comment['data']
                comments.append({
                    'url': url,
                    'author': comment_data.get('author'),
                    'date': datetime.utcfromtimestamp(comment_data.get('created_utc')).strftime('%Y-%m-%d'),
                    'timestamp': datetime.utcfromtimestamp(comment_data.get('created_utc')).strftime('%Y-%m-%d %H:%M:%S'),
                    'score': comment_data.get('score'),
                    'upvotes': comment_data.get('ups'),  # Reddit provides "ups" for upvotes
                    'downvotes': comment_data.get('downs'),  # Reddit provides "downs" for downvotes
                    'golds': comment_data.get('gilded'),
                    'comment': comment_data.get('body'),
                    'comment_id': comment_data.get('id'),
                    'subreddit': subreddit
                })
        return pd.DataFrame(comments)
    except Exception as e:
        print(f"Error processing comments for post {post_id}: {e}")
        return pd.DataFrame()



#### loading reddits and ids 

In [3]:
posts=pd.read_csv('all_posts/all.csv',usecols=['subreddit','id'])
posts.dropna(inplace=True)
posts.drop_duplicates(inplace=True)

posts.set_index(pd.Index(range(len(posts))),inplace=True)
posts

C:\Users\hp\AppData\Local\Temp\ipykernel_20668\157953981.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  posts=pd.read_csv('all_posts/all.csv',usecols=['subreddit','id'])


,subreddit,id
0,artificial,1h0atvn
1,artificial,1h082d4
2,artificial,1h04zzm
3,artificial,1gzkign
4,artificial,1gzksul
...,...,...
3280,worldnews,1gznr81
3281,worldnews,1gzt5vc
3282,worldnews,1gza6p5
3283,worldnews,1gzx8ro


## scraping comments

## جرب تعمل اكونت تاني 

In [6]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from functools import partial
def fetch_and_save_comments(headers, subreddit, post_id):
    """
    Fetch comments for a single post and return a DataFrame.
    """
    comments_df = fetch_comments(headers, subreddit, post_id)
    if not comments_df.empty:
        return comments_df
    return pd.DataFrame()  # Return an empty DataFrame if no comments were fetched

def main():
    # Authenticate to get headers
    headers = authenticate()

    # List of post IDs (replace 'posts' with your actual DataFrame)
    # Create a ThreadPoolExecutor
    all_comments = pd.DataFrame()
    with ThreadPoolExecutor(max_workers=5) as executor:  # Adjust `max_workers` as needed
        # Use partial to pass headers as a constant argument
        fetch_partial = partial(fetch_and_save_comments, headers)
        
        # Submit tasks to the executor
        futures = {
            executor.submit(fetch_partial, subreddit, post_id): (subreddit, post_id)
            for index, subreddit, post_id in posts.itertuples()
        }

        # Collect results as they complete
        for future in futures:
            try:
                comments_df = future.result()
                if not comments_df.empty:
                    all_comments = pd.concat([all_comments, comments_df], ignore_index=True)
            except Exception as e:
                print(f"Error fetching comments for {futures[future]}: {e}")

    # Save all comments to a CSV file
    all_comments_path = "comments_posts/comments_v2.csv"
    all_comments.to_csv(all_comments_path, index=False)
    print(f"Comments saved to {all_comments_path}")

if __name__ == "__main__":
    main()


3285it [00:00, 41591.07it/s]


Error processing comments for post 1guye4n: an integer is required (got type NoneType)
Error processing comments for post 1h00jww: an integer is required (got type NoneType)
Error processing comments for post 1h05r1f: an integer is required (got type NoneType)
Error processing comments for post 1h040ur: an integer is required (got type NoneType)
Error processing comments for post 1h04owv: an integer is required (got type NoneType)
Error processing comments for post 1h0btno: an integer is required (got type NoneType)
Error processing comments for post 1h00xlo: an integer is required (got type NoneType)
Error processing comments for post 1h0adcs: an integer is required (got type NoneType)
Error processing comments for post 1gzm1n6: an integer is required (got type NoneType)
Error processing comments for post 1h06pjf: an integer is required (got type NoneType)
Error processing comments for post 1gzlr2t: an integer is required (got type NoneType)
Error processing comments for post 1h0asu6: